# Batch Scoring Pipeline - Late Delivery Predictions for Open Deliveries

**Goal:** Generate late delivery risk predictions for **open deliveries** (not yet shipped).

**Use Case:** Enable operations team to:
- Identify deliveries at high risk of shipping late
- Prioritize corrective actions for strategic accounts
- Proactively communicate with business teams about potential delays

**Workflow:**
1. Load trained regression model from MLflow
2. Get **open deliveries** using DAX (deliveries without GI Date)
3. Generate predictions: days late + risk score + lateness bucket
4. Save predictions to Lakehouse table: `late_delivery_predictions`
5. Visualize high-risk deliveries
6. Enable Power BI reporting

### 🟦 1. Import Libraries

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Model loading
import mlflow

# Semantic Link - Connect to Power BI
import sempy.fabric as fabric

# Spark for writing to Lakehouse
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ All libraries imported")

### 🟦 2. Configuration

In [ ]:
# Semantic model name
DATASET = "DLV Aging Columns & Measures"

# MLflow model details
MODEL_NAME = "POC-LateDelivery-Regression-AutoML"
MODEL_VERSION = "latest"  # or specify version like "1", "2", etc.

# Target variable (must match training notebook)
TARGET_COLUMN = "AGE_REQ_DATE"

# Output table name
OUTPUT_TABLE = "late_delivery_predictions"

# Workspace
ws = fabric.get_workspace_id()

print(f"📊 Dataset: {DATASET}")
print(f"🤖 Model: {MODEL_NAME}")
print(f"📌 Version: {MODEL_VERSION}")
print(f"🎯 Target: {TARGET_COLUMN}")
print(f"💾 Output Table: {OUTPUT_TABLE}")

### 🟦 3. Load Trained Model from MLflow

In [ ]:
# Load model from MLflow registry
model_uri = f"models:/{MODEL_NAME}/{MODEL_VERSION}"

print(f"Loading model from: {model_uri}")
model = mlflow.sklearn.load_model(model_uri)

print(f"✅ Model loaded successfully!")
print(f"   Model type: {type(model).__name__}")

### 🟦 4. Load Open Deliveries from Semantic Model

Load **only open deliveries** (not yet shipped) that need predictions.

In [ ]:
# Load open deliveries (no GI Date = not shipped yet)
dax_query = """
EVALUATE
FILTER(
    Aging,
    ISBLANK(Aging[GI Date])
)
"""

print("Loading open deliveries...")
df_open = fabric.evaluate_dax(dataset=DATASET, dax_string=dax_query, workspace=ws)

# Clean column names
df_open.columns = [col.split('[')[-1].replace(']', '') if '[' in col else col for col in df_open.columns]

print(f"✅ Loaded {len(df_open):,} open deliveries")
df_open.head()

### 🟦 5. Prepare Features

Extract the same features used during training.

In [ ]:
# Define features (must match training notebook)
potential_features = [
    'Plant', 'Shipping Point', 'EWM Carrier Code',
    'Brand', 'Channel', 'Product Category', 'Product Type', 'Standard Or Custom',
    'STRATEGIC_ACCOUNT', 'Sold To - Key',
    'Delivery Type', 'DELIVERY_QTY', 'DELIVERY_VALUE_USD', 'Delivery Priority', 'Shipping Condition',
    'Credit Status', 'Distribution Status', 'STATUS'
]

# Add temporal features
if 'Delivery Created On' in df_open.columns:
    try:
        df_open['created_dayofweek'] = pd.to_datetime(df_open['Delivery Created On']).dt.dayofweek
        df_open['created_month'] = pd.to_datetime(df_open['Delivery Created On']).dt.month
        potential_features.extend(['created_dayofweek', 'created_month'])
    except:
        pass

# Filter to available features
feature_cols = [f for f in potential_features if f in df_open.columns]
print(f"✅ Using {len(feature_cols)} features")

### 🟦 6. Feature Engineering

Apply the same transformations used in training.

In [ ]:
# Prepare features
X = df_open[feature_cols].copy()

# Encode categoricals
categorical_cols = X.select_dtypes(include=['object', 'string']).columns.tolist()
for col in categorical_cols:
    X[col] = X[col].fillna('Unknown')
    X[col] = X[col].astype("category").cat.codes

# Fill numeric NaNs
numeric_cols = X.select_dtypes(include=['number']).columns.tolist()
for col in numeric_cols:
    if X[col].isnull().sum() > 0:
        X[col] = X[col].fillna(X[col].median())

print(f"✅ Features prepared: {X.shape}")

### 🟦 7. Generate Predictions

In [ ]:
# Generate predictions
predicted_days_late = model.predict(X)

print(f"✅ Generated {len(predicted_days_late):,} predictions")
print(f"Mean predicted days late: {predicted_days_late.mean():.2f}")
print(f"Late deliveries predicted: {(predicted_days_late > 0).sum():,}")

### 🟦 8. Create Output Dataset

In [ ]:
# Create output dataframe
from datetime import datetime

output_df = pd.DataFrame()

# Add delivery number
if 'Delivery Number' in df_open.columns:
    output_df['Delivery_Number'] = df_open['Delivery Number']

# Add key business columns
key_columns = ['Plant', 'Brand', 'Channel', 'Product Category', 'STRATEGIC_ACCOUNT',
               'DELIVERY_QTY', 'DELIVERY_VALUE_USD', 'STATUS']

for col in key_columns:
    if col in df_open.columns:
        output_df[col.replace(' ', '_')] = df_open[col]

# Add predictions
output_df['predicted_days_late'] = predicted_days_late
output_df['is_late'] = (predicted_days_late > 0).astype(int)

# Add lateness buckets
def bucket_lateness(days):
    if days <= 0: return 'On-time/Early'
    elif days <= 2: return '0-2 days late'
    elif days <= 5: return '3-5 days late'
    elif days <= 9: return '6-9 days late'
    else: return '10+ days late'

output_df['lateness_bucket'] = predicted_days_late.apply(bucket_lateness)

# Add risk score (0-1)
max_late = predicted_days_late.max()
min_late = predicted_days_late.min()
output_df['risk_score'] = ((predicted_days_late - min_late) / (max_late - min_late)).clip(0, 1)

# Add metadata
output_df['prediction_date'] = datetime.now()
output_df['model_name'] = MODEL_NAME

print(f"✅ Output created: {len(output_df):,} rows, {output_df.shape[1]} columns")

### 🟦 9. Save Predictions to Lakehouse

Write predictions to a Delta table for Power BI consumption.

In [ ]:
# Save to Lakehouse
spark_df = spark.createDataFrame(output_df)

print(f"💾 Writing to Lakehouse table: {OUTPUT_TABLE}")

spark_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(OUTPUT_TABLE)

print(f"✅ Saved {len(output_df):,} predictions")
print(f"   Predicted LATE: {output_df['is_late'].sum():,}")
print(f"   Predicted ON-TIME: {(~output_df['is_late'].astype(bool)).sum():,}")

### 🟦 10. Summary

In [ ]:
print("="*60)
print("SCORING COMPLETE")
print("="*60)

# Bucket summary
print("\n📦 Lateness Buckets:")
print(output_df['lateness_bucket'].value_counts().sort_index())

# Top 10 high risk
print("\n🚨 TOP 10 HIGHEST RISK:")
top_risk = output_df.nlargest(10, 'risk_score')[
    ['Delivery_Number', 'Brand', 'STRATEGIC_ACCOUNT', 
     'predicted_days_late', 'lateness_bucket', 'risk_score']
]
print(top_risk.to_string(index=False))

print("\n" + "="*60)
print(f"✅ Table '{OUTPUT_TABLE}' ready for Power BI")
print("="*60)

---

## ✅ Predictions Complete!

The `late_delivery_predictions` table is now available in your Lakehouse and can be used in Power BI for reporting and dashboards.